In [12]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

"""
STEP 0 >> imports; def clean_census & other functions
"""

# default cleaning method until proven otherwise
def clean_census_frame(csv_path , head=False , reset=True , set_index=False ):
    '''
    inputs) 
        >> csv_path
            > path to csv
        >> head
            > default=False
                >> if != False
                    > integer
                        >> returns the first {head} rows (using .head() method) 
                            > instead of enitre dataframe
        >> reset
            > default=True
                >> resets index after taking out rows
            > if set to False
                >> will not reset index
        >> set_index
            > default=False
            > if != False
                >> will set_index of new df to set_index
    output)
        >> dataframe cleaned like 2000 Census age&sex by 5-digit Zip Code (also how 2010 for same is cleaned)
    how)
        1. reads in csv , assumes it's large
        2. makes a copy for editing 
            > and potential future use
        3. locates readable column names  and non-readable names 
            > readable
                    > e.g. Estimate; SEX AND AGE - Total population
                >> assumes they are currently in row 0
            > non-readable
                    > e.g. HC01_VC03
                >> assumes they are currently == dataframe.columns
        4. replaces dataframe.columns (non-readable) with readable column names
            > and drops the old 0th column (column where readable names were stored)
        
    '''
    # load data
    df = pd.read_csv( csv_path , low_memory=False )

    # and copy
    _df = df.copy()

    # reset column names to current 0th row values
    _df.columns = _df.iloc[0]
    # new 2000 dataframe without row where values are from
    clean_df = _df[1:]
    
    # default
    if reset==True:
        # reset index
        clean_df = clean_df.reset_index()
        
    # set_index
    if set_index:
        clean_df = clean_df.set_index(set_index)
    
    if head:
        # return first {head} rows of dataframe
        return clean_df.head(head)
    else:
        # return dataframe
        return clean_df

'''
STEP 1 >> load data, reset; make copies
'''
def load_copy_data(i):
    '''
    loads data
    
    input)
        >> i
            > if 0
                >> .reset_index() after deleting row contining column names
            > if 1
                >> do not .reset_index()
                
    '''
    if i==0:
        # load with reset
        # 2011 
        twenty_eleven = clean_census_frame('../../data/American_Community_Survey/aff_download/ACS_11_5YR_DP05_with_ann.csv')
        # 2012
        # twenty_twelve = clean_census_frame('../../data/American_Community_Survey/aff_download/ACS_12_5YR_DP05_with_ann.csv')
        #2013
        # twenty_thirteen = clean_census_frame('../../data/American_Community_Survey/aff_download/ACS_13_5YR_DP05_with_ann.csv')
        # 2014
        # twenty_fourteen = clean_census_frame('../../data/American_Community_Survey/aff_download/ACS_14_5YR_DP05_with_ann.csv')
        # 2015
        # twenty_fifteen = clean_census_frame('../../data/American_Community_Survey/aff_download/ACS_15_5YR_DP05_with_ann.csv')
        #2016
        # twenty_sixteen = clean_census_frame('../../data/American_Community_Survey/aff_download/ACS_16_5YR_DP05_with_ann.csv')
        #2017
        # twenty_seventeen = clean_census_frame('../../data/American_Community_Survey/aff_download/ACS_17_5YR_DP05_with_ann.csv')
    if i==1:
        # load without reset
        # 2011 
        twenty_eleven = clean_census_frame('../../data/American_Community_Survey/aff_download/ACS_11_5YR_DP05_with_ann.csv',reset=False)
        # 2012
        # twenty_twelve = clean_census_frame('../../data/American_Community_Survey/aff_download/ACS_12_5YR_DP05_with_ann.csv',reset=False)
        #2013
        # twenty_thirteen = clean_census_frame('../../data/American_Community_Survey/aff_download/ACS_13_5YR_DP05_with_ann.csv',reset=False)
        # 2014
        # twenty_fourteen = clean_census_frame('../../data/American_Community_Survey/aff_download/ACS_14_5YR_DP05_with_ann.csv',reset=False)
        # 2015
        # twenty_fifteen = clean_census_frame('../../data/American_Community_Survey/aff_download/ACS_15_5YR_DP05_with_ann.csv',reset=False)
        #2016
        # twenty_sixteen = clean_census_frame('../../data/American_Community_Survey/aff_download/ACS_16_5YR_DP05_with_ann.csv',reset=False)
        #2017
        # twenty_seventeen = clean_census_frame('../../data/American_Community_Survey/aff_download/ACS_17_5YR_DP05_with_ann.csv',reset=False)
    
    # copy 
    # 2011 
    _y2k11 = twenty_eleven.copy()
    # 2012
    # _y2k12 = twenty_twelve.copy()
    #2013
    # _y2k13 = twenty_thirteen.copy()
    # 2014
    # _y2k14 = twenty_fourteen.copy()
    # 2015
    # _y2k15 = twenty_fifteen.copy()
    #2016
    # _y2k16 = twenty_sixteen.copy()
    #2017
    # _y2k17 = twenty_seventeen.copy()
    
    # output list of copied frames
    return _y2k11  # [_y2k11,_y2k12,_y2k13,_y2k14,_y2k15,_y2k16,_y2k17]


def test_non_unique(column_names):
    '''
    input) 
        >> list of column names {column_names}
            > columns to check for duplicate instances
    output)
        >> indexed list of names occouring more than once 
    '''
    # store first instance
    first_occour = []
    # store 2nd+ instance(s)
    non_unique = []
    # we're going to want index
    for i,_ in enumerate(column_names):
        # not first time
        if _ not in first_occour:
            first_occour.append(_)
        # if not first, tag&bag
        else:
            non_unique.append([i,_])
    # output index w/ non-first instances
    return non_unique


def to_numeric_but(save_these_columns,dataframe):
    '''
    split into 2 df and rejoin after convert to int
    
    inputs:
        >> save_these_columns=number of columns to save
            > currently must include one end of df 
                >> might could run function multiple times to edit slices
                >> single number, not range (yet)
        >> dataframe
            > dataframe to shif to numeric (but)
    output:
        >> concatted pd.DataFrame of 
            > og columns you chose to save
            > columns converted to numeric
    '''
    # copy df for editing
    k = dataframe.copy()

    # columns to save
    save_k = k[k.columns[:save_these_columns]]
    # columns to edit
    switch_k = k[k.columns[save_these_columns:]]

    # edited columns  # coerce , ignore , raise
    swapped_k = switch_k.apply(pd.to_numeric, errors='coerce')

    # new (edited) dataframe
    new_k = pd.concat([save_k,switch_k],axis=1)

    return new_k


def geography_to_zipcode_ids_to_numeric(dataframe):
    '''
    convert 
        >> .Geography values 
            > like 'ZCTA5 00601' 
            > to int(00601)
        >> .Id values
            > like '8600000US00601' 
            > to int(860000000601)
        >> .Id2 values
            > like '00601'
            > to int(00601)
    '''
    # copy
    df = dataframe.copy()
    
    # set old Geography
    geo = df.Geography
    # set old Id
    _id = df.Id
    # set old Id2
    __id2 = df.Id2
    
    # make new 'Geography' values
    new_geos = [int(i[-5:]) for i in geo]
    # new 'Id' values
    new_id = [int(''.join(i.split('US'))) for i in _id]
    # new .Id2 instances
    new__id2 = [int(d) for d in __id2]
    
    # convert dataframe
    new_df = df.copy()
    new_df.Geography = new_geos
    new_df.Id = new_id
    new_df.Id2 = new__id2
    
    # return new df
    return new_df

In [13]:
# load w/o reset
f = load_copy_data(1)

In [14]:
# copy for safeguard and hedge reload
frames = f.copy()

In [15]:
# extract copy of 2011 
y2k11 = frames.copy()  #[0]

In [16]:
# examine (2011)
y2k11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33120 entries, 1 to 33120
Columns: 327 entries, Id to Percent Margin of Error; HISPANIC OR LATINO AND RACE - Total housing units
dtypes: object(327)
memory usage: 82.6+ MB


In [25]:
nuy11 = test_non_unique(y2k11)
len(nuy11)

32

In [18]:
# convert all but first 3 columns to numeric
data = y2k11.copy()
k2011 = to_numeric_but(save_these_columns=3,dataframe=data)

In [19]:
k2011.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33120 entries, 1 to 33120
Columns: 407 entries, Id to Percent Margin of Error; HISPANIC OR LATINO AND RACE - Total housing units
dtypes: object(407)
memory usage: 102.8+ MB


In [26]:
nonuni = test_non_unique(k2011)
len(nonuni)

112

In [34]:
nuy11[:2] , nonuni[:2]

([[87, 'Estimate; SEX AND AGE - 18 years and over'],
  [88, 'Margin of Error; SEX AND AGE - 18 years and over']],
 [[8, 'Estimate; SEX AND AGE - Male'], [9, 'Estimate; SEX AND AGE - Male']])

In [54]:
len(set(k2011.columns[:10]))

9

In [11]:
for i in k2011.columns:
    if i not in y2k11.columns:
        print(i)

In [55]:
# now convert the first 3 columns
adjust_first_3 = k2011.copy()
_2011df_ = geography_to_zipcode_ids_to_numeric(adjust_first_3)

In [56]:
_2011df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33120 entries, 1 to 33120
Columns: 407 entries, Id to Percent Margin of Error; HISPANIC OR LATINO AND RACE - Total housing units
dtypes: int64(3), object(404)
memory usage: 102.8+ MB


In [117]:
_2011df_ = _2011df_.dropna(axis=1)  # , how='any'

In [118]:
# no change, likely due to 'coerce' instead of 'ignore'
_2011df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33120 entries, 1 to 33120
Columns: 407 entries, Id to Percent Margin of Error; HISPANIC OR LATINO AND RACE - Total housing units
dtypes: int64(3), object(404)
memory usage: 102.8+ MB


In [119]:
_2011df_.head(100)

,Id,Id2,Geography,Estimate; SEX AND AGE - Total population,Margin of Error; SEX AND AGE - Total population,Percent; SEX AND AGE - Total population,Percent Margin of Error; SEX AND AGE - Total population,Estimate; SEX AND AGE - Male,Estimate; SEX AND AGE - Male,Estimate; SEX AND AGE - Male,...,Percent; HISPANIC OR LATINO AND RACE - Not Hispanic or Latino - Two or more races - Two races including Some other race,Percent Margin of Error; HISPANIC OR LATINO AND RACE - Not Hispanic or Latino - Two or more races - Two races including Some other race,"Estimate; HISPANIC OR LATINO AND RACE - Not Hispanic or Latino - Two or more races - Two races excluding Some other race, and Three or more races","Margin of Error; HISPANIC OR LATINO AND RACE - Not Hispanic or Latino - Two or more races - Two races excluding Some other race, and Three or more races","Percent; HISPANIC OR LATINO AND RACE - Not Hispanic or Latino - Two or more races - Two races excluding Some other race, and Three or more races","Percent Margin of Error; HISPANIC OR LATINO AND RACE - Not Hispanic or Latino - Two or more races - Two races excluding Some other race, and Three or more races",Estimate; HISPANIC OR LATINO AND RACE - Total housing units,Margin of Error; HISPANIC OR LATINO AND RACE - Total housing units,Percent; HISPANIC OR LATINO AND RACE - Total housing units,Percent Margin of Error; HISPANIC OR LATINO AND RACE - Total housing units
1,860000000601,601,601,18533,310,18533,(X),8971,6495,1080,...,0.0,0.2,0,92,0.0,0.2,6503,164,(X),(X)
2,860000000602,602,602,41930,136,41930,(X),20568,15453,2395,...,0.0,0.1,228,125,0.5,0.3,16336,225,(X),(X)
3,860000000603,603,603,54475,839,54475,(X),26588,19831,3725,...,0.0,0.1,250,130,0.5,0.2,23245,334,(X),(X)
4,860000000606,606,606,6386,291,6386,(X),3085,2206,337,...,0.0,0.5,0,92,0.0,0.5,2373,111,(X),(X)
5,860000000610,610,610,29111,173,29111,(X),14162,10512,1880,...,0.0,0.1,0,92,0.0,0.1,11308,147,(X),(X)
6,860000000612,612,612,70541,1436,70541,(X),33449,24810,4914,...,0.0,0.1,40,46,0.1,0.1,29575,452,(X),(X)
7,860000000616,616,616,10617,866,10617,(X),5166,3723,804,...,0.0,0.3,3,6,0.0,0.1,4466,264,(X),(X)
8,860000000617,617,617,24458,261,24458,(X),11752,8535,1537,...,0.0,0.1,17,28,0.1,0.1,9627,192,(X),(X)
9,860000000622,622,622,5419,854,5419,(X),2392,2030,596,...,0.0,0.6,0,92,0.0,0.6,7042,351,(X),(X)
10,860000000623,623,623,45314,854,45314,(X),21904,16161,3346,...,0.0,0.1,0,92,0.0,0.1,19997,390,(X),(X)


In [131]:
# for i,j in _2011df_.sample(5,axis=0):
#     print(row)
x=-1
out=[]
for i in _2011df_.sample(1000,axis=0).values:
    x+=1
    y=0
    for j in i:
        y+=1
        if j == '(X)':
            out.append((x,y))

In [132]:
collectj = []
for i,j in out: 
    collectj.append(j)

In [133]:
count_out=[]
for n in set(collectj):
    q = collectj.count(n)
    count_out.append((n,q))

In [134]:
pd.DataFrame(data=count_out,columns=['column','count'])

,column,count
0,407,1000
1,7,1000
2,111,1000
3,179,1000
4,119,1000
5,86,1000
6,87,1000
7,151,1000
8,406,1000
9,315,1000


- ***notes***:
    - that wasn't too hard
- ***actions***:
    - drop these columns

In [138]:
drop_these = [i for i,j in count_out]
keep_these = [n for n in range(len(_2011df_.columns)) if n not in drop_these]

In [139]:
keep_these

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 

In [140]:
keep_these = _2011df_.columns[[keep_these]]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexes/base.py:3968: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


In [145]:
final_2011 = _2011df_.copy()
final_2011 = final_2011[keep_these]

In [146]:
final_2011

,Id,Id2,Geography,Estimate; SEX AND AGE - Total population,Margin of Error; SEX AND AGE - Total population,Percent; SEX AND AGE - Total population,Percent Margin of Error; SEX AND AGE - Total population,Estimate; SEX AND AGE - Male,Estimate; SEX AND AGE - Male,Estimate; SEX AND AGE - Male,...,Margin of Error; HISPANIC OR LATINO AND RACE - Not Hispanic or Latino - Two or more races - Two races including Some other race,Percent; HISPANIC OR LATINO AND RACE - Not Hispanic or Latino - Two or more races - Two races including Some other race,Percent Margin of Error; HISPANIC OR LATINO AND RACE - Not Hispanic or Latino - Two or more races - Two races including Some other race,"Estimate; HISPANIC OR LATINO AND RACE - Not Hispanic or Latino - Two or more races - Two races excluding Some other race, and Three or more races","Margin of Error; HISPANIC OR LATINO AND RACE - Not Hispanic or Latino - Two or more races - Two races excluding Some other race, and Three or more races","Percent; HISPANIC OR LATINO AND RACE - Not Hispanic or Latino - Two or more races - Two races excluding Some other race, and Three or more races","Percent Margin of Error; HISPANIC OR LATINO AND RACE - Not Hispanic or Latino - Two or more races - Two races excluding Some other race, and Three or more races",Estimate; HISPANIC OR LATINO AND RACE - Total housing units,Margin of Error; HISPANIC OR LATINO AND RACE - Total housing units,Percent; HISPANIC OR LATINO AND RACE - Total housing units
1,860000000601,601,601,18533,310,18533,(X),8971,6495,1080,...,92,0.0,0.2,0,92,0.0,0.2,6503,164,(X)
2,860000000602,602,602,41930,136,41930,(X),20568,15453,2395,...,92,0.0,0.1,228,125,0.5,0.3,16336,225,(X)
3,860000000603,603,603,54475,839,54475,(X),26588,19831,3725,...,92,0.0,0.1,250,130,0.5,0.2,23245,334,(X)
4,860000000606,606,606,6386,291,6386,(X),3085,2206,337,...,92,0.0,0.5,0,92,0.0,0.5,2373,111,(X)
5,860000000610,610,610,29111,173,29111,(X),14162,10512,1880,...,9,0.0,0.1,0,92,0.0,0.1,11308,147,(X)
6,860000000612,612,612,70541,1436,70541,(X),33449,24810,4914,...,92,0.0,0.1,40,46,0.1,0.1,29575,452,(X)
7,860000000616,616,616,10617,866,10617,(X),5166,3723,804,...,92,0.0,0.3,3,6,0.0,0.1,4466,264,(X)
8,860000000617,617,617,24458,261,24458,(X),11752,8535,1537,...,92,0.0,0.1,17,28,0.1,0.1,9627,192,(X)
9,860000000622,622,622,5419,854,5419,(X),2392,2030,596,...,92,0.0,0.6,0,92,0.0,0.6,7042,351,(X)
10,860000000623,623,623,45314,854,45314,(X),21904,16161,3346,...,92,0.0,0.1,0,92,0.0,0.1,19997,390,(X)


In [67]:
_2011df_.Geography.sample(1000)

8752     28128
30392    92107
13100    40316
20290    59636
31933    97108
29207    85346
3257     12424
929       3809
14707    45360
18772    55933
21802    63005
16380    49321
9668     30157
9766     30332
28106    80917
27577    79248
19672    57793
12585    38702
26782    77362
30084    90221
29976    89460
3276     12448
153       1034
5864     18830
32977    99649
17828    53529
24346    70722
15952    48393
13290    41064
2797     10988
         ...  
9661     30144
20474    60088
8921     28467
6491     21040
17189    51101
3981     14025
2713     10577
25269    73068
24765    72019
22702    65627
31941    97116
31899    97037
32118    97469
3468     12836
29711    88024
9335     29453
1377      4930
16551    49725
8551     27801
2993     11742
6787     22025
4392     15054
25929    75119
4086     14227
2124      7607
21947    63388
17038    50595
7783     25090
14456    44707
12922    39669
Name: Geography, Length: 1000, dtype: int64

In [61]:
# Convert DataFrame to matrix
mat = _2011df_.values
# Using sklearn
km = KMeans(n_clusters=5)
km.fit(mat)
# Get cluster assignment labels
labels = km.labels_
# Format results as a DataFrame
results = pandas.DataFrame([dataset.index,labels]).T

ValueError: could not convert string to float: '(X)'

In [ ]:
# copy df for editing
k11 = y2k11.copy()

'''split into 2 df and rejoin after convert to int'''
# df to save
save_k11 = k11.copy()
# columns to save
save_k11 = save_k11.copy()[save_k11.columns[:3]]

# df to edit
switch_k11 = k11.copy()
# columns to edit
switch_k11 = switch_k11.copy()[switch_k11.columns[3:]]

# edited columns
swapped_k11 = switch_k11.copy().apply(pd.to_numeric, errors='ignore')

# new (edited) dataframe
new_k11 = pd.concat([save_k11,swapped_k11],axis=1)

len(new_k11.columns)

In [ ]:
# # pull column names
# k11_cols = y2k11.copy().columns[4:]
# for c in range(len(k11_cols)):
#     a = pd.Series(k11[k11_cols])
# #     k11.loc[[k11_cols][c]] = pd.to_numeric(a,errors='ignore')
# # pd.Series(k11[k11_cols[0]])

In [ ]:
k11_cols = y2k11.copy().columns
__k11_cols__ = y2k11.copy().columns[4:]
# len(k11_cols),len(__k11_cols__)
print(f'{__k11_cols__[:3]}\n{k11_cols[:3]}')

In [ ]:
for frame in frames:
    print(frame.info(),'\n')

In [ ]:
copies = frames[:2].copy()
for df in copies:
    for column in df.columns: 
        df[column] = pd.to_numeric([df[column]], errors='ignore')

- ***NOTE***:
    - path to mvp
        - whiteboard_pics/acs_5yr_11-17_path-to-mvp.jpg

In [ ]:
dataset_array = frames[0].values
dataset_array

In [ ]:
'''
STEP 2 >> find all columns which coexist across all dataframes at current position
'''
columns_by_frame = [frame.columns for frame in frames]
count_columns_by_frame = [len(frame) for frame in columns_by_frame]

In [ ]:
out = []
# for range of df with most columns
for count in range(len(max(count_columns_by_frame))):
    # if index of every frame is same as index of frame with most columns
    if [frame for frame in columns_by_frame][count] == frames[6][count]:
        out.append(count)
out

In [ ]:
# collect all
years = frames  # [y2k11,y2k12,y2k13,y2k14,y2k15,y2k16,y2k17]
for year in years:
    print(len(year.columns),'\n',year.info(),'\n\n')

In [ ]:


X = np.array([[1, 2], [1, 4], [1, 0], [10, 2], [10, 4], [10, 0]])

kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
kmeans.labels_

# array([1, 1, 1, 0, 0, 0], dtype=int32)
kmeans.predict([[0, 0], [12, 3]])
# array([1, 0], dtype=int32)
kmeans.cluster_centers_
# array([[10.,  2.], [ 1.,  2.]])

In [ ]:


# Scikit learn plays really well with Pandas, so I suggest you use it. Here's an example:

# In [1]: 
# import pandas as pd
# import numpy as np
# from sklearn.cross_validation import train_test_split
data = np.reshape(np.random.randn(20),(10,2)) # 10 training examples
labels = np.random.randint(2, size=10) # 10 labels

# In [2]: 
X = pd.DataFrame(data)
y = pd.Series(labels)

# In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    random_state=0)

# In [4]: X_test
# Out[4]:

#      0       1
# 2   -1.39   -1.86
# 8    0.48   -0.81
# 4   -0.10   -1.83

# In [5]: y_test
# Out[5]:

# 2    1
# 8    1
# 4    1

In [ ]:
X_train

In [ ]:
# q=pd.read_csv('../../data/American_Community_Survey/aff_download/ACS_11_5YR_DP05_with_ann.csv')

In [ ]:
pizfsdazapizzaadsf = (2,1,0,4,32,7,2,9,5)
max(pizfsdazapizzaadsf)